In [1]:
import pandas as pd
import numpy as np
%pylab inline

Populating the interactive namespace from numpy and matplotlib


## Topics

In [2]:
docs_topics = pd.read_csv("../download/documents_topics.csv")
docs_topics.head()

,document_id,topic_id,confidence_level
0,1595802,140,0.073113
1,1595802,16,0.059416
2,1595802,143,0.045421
3,1595802,170,0.038867
4,1524246,113,0.196450


In [3]:
docs_topics['value']=docs_topics['confidence_level']
docs_topics['value2']=np.square(docs_topics['value'])
docs_topics.drop(['confidence_level'], axis=1, inplace=True)
docs_topics.head()

,document_id,topic_id,value,value2
0,1595802,140,0.073113,0.005346
1,1595802,16,0.059416,0.003530
2,1595802,143,0.045421,0.002063
3,1595802,170,0.038867,0.001511
4,1524246,113,0.196450,0.038593


In [4]:
docs_norms2 = pd.DataFrame({'norm2':docs_topics.groupby(['document_id'])['value2'].sum()}).reset_index()
docs_topics.drop(['value2'], axis=1, inplace=True)
docs_norms2.head()

,document_id,norm2
0,1,0.000926
1,2,0.000170
2,3,0.003571
3,4,0.024792
4,5,0.027574


### Train

In [ ]:
page_addoc_dist = pd.read_csv("../generated/final/events_clicks_CTR_addocs_train.csv", usecols=['document_id', 'addoc_id'])
page_addoc_dist.count()

In [ ]:
page_addoc_dist = pd.DataFrame({'size':page_addoc_dist.groupby(['document_id', 'addoc_id']).size()}).reset_index().drop(['size'], axis=1)
page_addoc_dist.count()

In [ ]:
len(page_addoc_dist.document_id.unique()), len(page_addoc_dist.addoc_id.unique())

In [ ]:
page_addoc_diffs = page_addoc_dist.merge(docs_topics, left_on='addoc_id', right_on='document_id')
page_addoc_diffs.rename(columns={'document_id_x':'document_id'}, inplace=True)
page_addoc_diffs.drop(['document_id_y'], axis=1, inplace=True)
page_addoc_diffs = page_addoc_diffs.merge(docs_topics, on=['document_id', 'topic_id'])
page_addoc_diffs['value_2xy'] = 2*page_addoc_diffs['value_x']*page_addoc_diffs['value_y']
page_addoc_diffs.drop(['topic_id', 'value_x', 'value_y'], axis=1, inplace=True)
page_addoc_diffs = pd.DataFrame({'value_2xy':page_addoc_diffs.groupby(['document_id', 'addoc_id'])['value_2xy'].sum()}).reset_index()

page_addoc_diffs.head()

In [ ]:
page_addoc_diffs.count()

In [ ]:
page_addoc_dist = page_addoc_dist.merge(docs_norms2, on=['document_id'])
page_addoc_dist = page_addoc_dist.merge(docs_norms2, left_on=['addoc_id'], right_on=['document_id'])
page_addoc_dist.rename(columns={'document_id_x':'document_id'}, inplace=True)
page_addoc_dist.drop(['document_id_y'], axis=1, inplace=True)
page_addoc_dist = page_addoc_dist.merge(page_addoc_diffs, on=['document_id', 'addoc_id'], how='left').fillna(0)
page_addoc_dist['dist'] = np.sqrt( page_addoc_dist['norm2_x'] + page_addoc_dist['norm2_y'] - page_addoc_dist['value_2xy'] )
page_addoc_dist.drop(['norm2_x', 'norm2_y', 'value_2xy'], axis=1, inplace=True)

page_addoc_dist.head()

In [ ]:
page_addoc_dist.to_csv("../generated/final/page_addoc_topics_no_w_dist.csv", index=False)

In [ ]:
page_addoc_dist.count()

### Test

In [5]:
page_addoc_dist = pd.read_csv("../generated/final/events_clicks_CTR_addocs_test.csv", usecols=['document_id', 'addoc_id'])
page_addoc_dist.count()

document_id    32225162
addoc_id       32225162
dtype: int64

In [6]:
page_addoc_dist = pd.DataFrame({'size':page_addoc_dist.groupby(['document_id', 'addoc_id']).size()}).reset_index().drop(['size'], axis=1)
page_addoc_dist.count()

document_id    13358295
addoc_id       13358295
dtype: int64

In [7]:
len(page_addoc_dist.document_id.unique()), len(page_addoc_dist.addoc_id.unique())

(502297, 132067)

In [8]:
page_addoc_diffs = page_addoc_dist.merge(docs_topics, left_on='addoc_id', right_on='document_id')
page_addoc_diffs.rename(columns={'document_id_x':'document_id'}, inplace=True)
page_addoc_diffs.drop(['document_id_y'], axis=1, inplace=True)
page_addoc_diffs = page_addoc_diffs.merge(docs_topics, on=['document_id', 'topic_id'])
page_addoc_diffs['value_2xy'] = 2*page_addoc_diffs['value_x']*page_addoc_diffs['value_y']
page_addoc_diffs.drop(['topic_id', 'value_x', 'value_y'], axis=1, inplace=True)
page_addoc_diffs = pd.DataFrame({'value_2xy':page_addoc_diffs.groupby(['document_id', 'addoc_id'])['value_2xy'].sum()}).reset_index()

page_addoc_diffs.head()

,document_id,addoc_id,value_2xy
0,8,988108,0.020826
1,8,1066278,0.004875
2,8,1087098,0.032166
3,8,1516087,0.015370
4,8,1578222,0.004595


In [9]:
page_addoc_diffs.count()

document_id    1718659
addoc_id       1718659
value_2xy      1718659
dtype: int64

In [10]:
page_addoc_dist = page_addoc_dist.merge(docs_norms2, on=['document_id'])
page_addoc_dist = page_addoc_dist.merge(docs_norms2, left_on=['addoc_id'], right_on=['document_id'])
page_addoc_dist.rename(columns={'document_id_x':'document_id'}, inplace=True)
page_addoc_dist.drop(['document_id_y'], axis=1, inplace=True)
page_addoc_dist = page_addoc_dist.merge(page_addoc_diffs, on=['document_id', 'addoc_id'], how='left').fillna(0)
page_addoc_dist['dist'] = np.sqrt( page_addoc_dist['norm2_x'] + page_addoc_dist['norm2_y'] - page_addoc_dist['value_2xy'] )
page_addoc_dist.drop(['norm2_x', 'norm2_y', 'value_2xy'], axis=1, inplace=True)

page_addoc_dist.head()

,document_id,addoc_id,dist
0,8,287704,0.131094
1,41490,287704,0.120471
2,57208,287704,0.145902
3,204133,287704,0.072116
4,297088,287704,0.133596


In [11]:
page_addoc_dist.to_csv("../generated/final/page_addoc_topics_no_w_dist_test.csv", index=False)

In [12]:
page_addoc_dist.count()

document_id    11616962
addoc_id       11616962
dist           11616962
dtype: int64